# Task6 连接

## 1 知识梳理（重点记忆）

### 1.1 关系型连接
- 连接方式主要有左连接`left`、右连接`right`、内连接`inner`、外连接`outer`
- 值连接通过`merge`函数实现，`on`为连接的键，`how`为连接方式
- 索引连接通过`join`函数实现，`on`为索引名，`how`为连接方式，`lsuffix`和`rsuffix`为左右后缀

### 1.2 方向连接
- `concat`函数：`axis`为连接方向，`join`为连接方式，`keys`为源表表名
- 序列与表合并，通过`append`函数（对行的添加）和`assign`函数（对列的添加，并返回DataFrame副本）

### 1.3 类连接操作
- `compare`函数：主要用于两表/两序列比较
- `combine`函数：主要用于组合，可传入自定义的规则函数，函数有两个参数`s1`,`s2`，分别来自于两张表对应的列

## 2 练一练

In [1]:
import numpy as np
import pandas as pd

### 2.1 第1题
上面以多列为键的例子中，错误写法显然是一种多对多连接，而正确写法是一对一连接，请修改原表，使得以多列为键的正确写法能够通过`validate='1:m'`的检验，但不能通过`validate='m:1'`的检验。

**我的解答：**

In [2]:
df1 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Age':[20, 21],
                    'Class':['one', 'two']})
df1

,Name,Age,Class
0,San Zhang,20,one
1,San Zhang,21,two


In [3]:
# 将df2中Class列数据修改为['one', 'one']，即一班有两个叫张三的人，一个男生一个女生
df2 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Gender':['F', 'M'],
                    'Class':['one', 'one']})
df2

,Name,Gender,Class
0,San Zhang,F,one
1,San Zhang,M,one


In [4]:
df1.merge(df2, on=['Name', 'Class'], how='left', validate="1:m")

,Name,Age,Class,Gender
0,San Zhang,20,one,F
1,San Zhang,20,one,M
2,San Zhang,21,two,NaN


In [5]:
try:
    df1.merge(df2, on=['Name', 'Class'], how='left', validate="m:1")
except Exception as e:
    print(e)

Merge keys are not unique in right dataset; not a many-to-one merge


### 2.2 第2题 
请在上述代码的基础上修改，保留`df2`中4个未被`df1`替换的相应位置原始值。

**我的解答：**

In [6]:
df1 = pd.DataFrame({'A':[1,2], 'B':[3,4], 'C':[5,6]})
df2 = pd.DataFrame({'B':[5,6], 'C':[7,8], 'D':[9,10]}, index=[1,2])

In [7]:
df1

,A,B,C
0,1,3,5
1,2,4,6


In [8]:
df2

,B,C,D
1,5,7,9
2,6,8,10


In [9]:
def choose_min(s1, s2):
    s2 = s2.reindex_like(s1)
    res = s1.where(s1<s2, s2)
    # isna表示是否为缺失值，返回布尔序列（删除该行即可）
    #res = res.mask(s1.isna())
    return res

df1.combine(df2, choose_min)

,A,B,C,D
0,NaN,NaN,NaN,NaN
1,NaN,4.0,6.0,9.0
2,NaN,6.0,8.0,10.0


### 2.3 第3题
除了`combine`之外，`pandas`中还有一个`combine_first`方法，其功能是在对两张表组合时，若第二张表中的值在第一张表中对应索引位置的值不是缺失状态，那么就使用第一张表的值填充。下面给出一个例子，请用`combine`函数完成相同的功能。

In [10]:
df1 = pd.DataFrame({'A':[1,2], 'B':[3,np.nan]})
df2 = pd.DataFrame({'A':[5,6], 'B':[7,8]}, index=[1,2])
df1.combine_first(df2)

,A,B
0,1.0,3.0
1,2.0,7.0
2,6.0,8.0


**我的解答：**

In [11]:
def choose_min(s1, s2):
    s2 = s2.reindex_like(s1)
    res = s1.mask(s1.isna(), s2)
    return res

df1.combine(df2, choose_min)

,A,B
0,1.0,3.0
1,2.0,7.0
2,6.0,8.0


## 3 练习

### 3.1 Ex1：美国疫情数据集

现有美国4月12日至11月16日的疫情报表，数据见`notebook/pandas20/data/us_report`文件夹路径，请将`New York`的`Confirmed, Deaths, Recovered, Active`合并为一张表，索引为按如下方法生成的日期字符串序列：

In [12]:
date = pd.date_range('20200412', '20201116').to_series()
date = date.dt.month.astype('string').str.zfill(2) +'-'+ date.dt.day.astype('string').str.zfill(2) +'-'+ '2020'
date = date.tolist()
date[:5]

['04-12-2020', '04-13-2020', '04-14-2020', '04-15-2020', '04-16-2020']

**我的解答：**

1. 先进行尝试，读取两个文件，最后通过`concat`函数进行合并

In [13]:
df1 = pd.read_csv('../data/us_report/04-12-2020.csv', usecols=['Confirmed', 'Deaths', 'Recovered', 'Active', 'Province_State'])
df2 = pd.read_csv('../data/us_report/04-13-2020.csv', usecols=['Confirmed', 'Deaths', 'Recovered', 'Active', 'Province_State'])
df1 = df1[df1['Province_State']=='New York']
df2 = df2[df2['Province_State']=='New York']

In [14]:
pd.concat([df1, df2])

,Province_State,Confirmed,Deaths,Recovered,Active
35,New York,189033,9385,23887.0,179648.0
36,New York,195749,10058,23887.0,185691.0


2. 整合代码，并修改索引列

In [15]:
concat_list = []

for d in date:
    df = pd.read_csv('../data/us_report/' + d + '.csv', usecols=['Confirmed', 'Deaths', 'Recovered', 'Active', 'Province_State'])
    df = df[df['Province_State']=='New York'].iloc[:,1:]
    concat_list.append(df)

res = pd.concat(concat_list)
res.index = date
res.head()

,Confirmed,Deaths,Recovered,Active
04-12-2020,189033,9385,23887.0,179648.0
04-13-2020,195749,10058,23887.0,185691.0
04-14-2020,203020,10842,23887.0,192178.0
04-15-2020,214454,11617,23887.0,202837.0
04-16-2020,223691,14832,23887.0,208859.0


### 3.2 Ex2：实现join函数

请实现带有`how`参数的`join`函数

* 假设连接的两表无公共列
* 调用方式为 `join(df1, df2, how="left")`
* 给出测试样例